<a href="https://colab.research.google.com/github/supai-red/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Andrea_Christelle_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import scipy.stats
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

import pandas as pd
import numpy as np

In [6]:
from google.colab import files 
uploaded = files.upload()

Saving house-votes-84.data to house-votes-84.data


In [7]:
df = pd.read_csv('house-votes-84.data', header=None)
print(df.shape)
df.head()

(435, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [8]:
#Adding column headers specified in attribute information at: https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records
df.columns = ["party", "handicapped_infants", "water_project_cost_sharing", "adoption_of_the_budget_resolution","physician_fee_freeze","el_salvador_aid","religious_groups_in_schools","anti_satellite_test_ban","aid_to_nicaraguan_contras","mx_missile","immigration","synfuels_corporation_cutback","education_spending","superfund_right_to_sue","crime","duty_free_exports","export_administration_act_south_africa" ]

df.head()
df.shape

(435, 17)

In [11]:
#Check missing values
df.isna().sum().sum()
#"?" counts as a value, yet provides no information.


0

###Make voting records numeric

In [12]:
df = df.replace({'?':np.NaN, 'n':0, 'y':1})
print(df.shape) 
df.head()


(435, 17)


,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [13]:
df.isnull().sum()

party                                       0
handicapped_infants                        12
water_project_cost_sharing                 48
adoption_of_the_budget_resolution          11
physician_fee_freeze                       11
el_salvador_aid                            15
religious_groups_in_schools                11
anti_satellite_test_ban                    14
aid_to_nicaraguan_contras                  15
mx_missile                                 22
immigration                                 7
synfuels_corporation_cutback               21
education_spending                         31
superfund_right_to_sue                     25
crime                                      17
duty_free_exports                          28
export_administration_act_south_africa    104
dtype: int64

###Defining Republicans

In [14]:
rep = df[df.party == "republican"]
print(rep.shape)
rep.head()

(168, 17)


,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


###Defining Democrats

In [15]:
dem = df[df.party == "democrat"]
print(dem.shape)
dem.head()

(267, 17)


,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [16]:
df.party.value_counts()

democrat      267
republican    168
Name: party, dtype: int64

## 1 Sample T-test

In [0]:
from scipy.stats import ttest_1samp
#used for 1 sample
dir(scipy.stats)

#used for more than 1 sample
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

### T-test part 1: null hypothesis 

There is 0 support for this bill among Republicans in the House.
Because there is one sample null hypothesis can be specified.

###T-test part 2: alternative hypothesis
There is non-0 support for this bill. (There is some support among Republicans in the house.)

In [18]:
print (rep["water_project_cost_sharing"].mean())
ttest_1samp(rep["water_project_cost_sharing"], 0, nan_policy="omit")
#Replace 0 with .5 to make the null hypothesis Republics are split evenly. 
####would like to review https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html

# Due to a t-statistic of 12.29, and a p-value of 2.53(**-24), reject
#the null hypothesis that there is 0 Republican support for the water_project
#cost sharing.  

0.5067567567567568


Ttest_1sampResult(statistic=12.28932045559371, pvalue=2.525482675130834e-24)

In [19]:
#T-test where the null hypothesis is Republican support is evenly divided.
# Alternative: Republican support is not even. 
ttest_1samp(rep["water_project_cost_sharing"], .5, nan_policy="omit")

Ttest_1sampResult(statistic=0.16385760607458383, pvalue=0.8700683158522193)

###T-test part 3: confidence level: 95%

Reject the null hypothesis when P-value < (1-confidence level)

In [0]:
##SAVING FOR LATER
# style.use('fivethirtyeight')

##2 Sample T-test

T-test part 1: null hypothesis - compares means
Democrats support more than Republicans

In [21]:
print ("Rep mean is", rep["water_project_cost_sharing"].mean())
print ("Dem mean is" ,dem["water_project_cost_sharing"].mean())

Rep mean is 0.5067567567567568
Dem mean is 0.502092050209205


In [0]:
ttest_ind(rep["water_project_cost_sharing"], dem["water_project_cost_sharing"], nan_policy="omit")

#write a function for 2 Sample T-tests on votes
def t_test_votes(a, b):
  ttest_ind(rep["water_project_cost_sharing"], dem["water_project_cost_sharing"], nan_policy="omit")


In [23]:
dem.describe()

,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
count,258.000000,239.000000,260.000000,259.000000,255.000000,258.000000,259.000000,263.000000,248.000000,263.000000,255.000000,249.000000,252.000000,257.000000,251.000000,185.000000
mean,0.604651,0.502092,0.888462,0.054054,0.215686,0.476744,0.772201,0.828897,0.758065,0.471483,0.505882,0.144578,0.289683,0.350195,0.637450,0.935135
std,0.489876,0.501045,0.315405,0.226562,0.412106,0.500430,0.420224,0.377317,0.429121,0.500138,0.500949,0.352383,0.454518,0.477962,0.481697,0.246956
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [58]:
def ttest(df1, df2, column_name):

  a = df1[column_name]
  b = df2[column_name]

  statistic, pvalue = ttest_ind(a, b, nan_policy="omit")
  print("The t-statistic is" ,  statistic, "The p-value is " , pvalue)

ttest (dem, rep, "handicapped_infants")


The t-statistic is 9.205264294809222 The p-value is  1.613440327937243e-18


In [45]:
print(df.columns)

Index(['party', 'handicapped_infants', 'water_project_cost_sharing',
       'adoption_of_the_budget_resolution', 'physician_fee_freeze',
       'el_salvador_aid', 'religious_groups_in_schools',
       'anti_satellite_test_ban', 'aid_to_nicaraguan_contras', 'mx_missile',
       'immigration', 'synfuels_corporation_cutback', 'education_spending',
       'superfund_right_to_sue', 'crime', 'duty_free_exports',
       'export_administration_act_south_africa'],
      dtype='object')


In [56]:
def ttest(df1, df2):
  for x in (df1.columns, df2.columns):
    statistic, pvalue = ttest_ind(df1[x], df2[x], nan_policy="omit")
    return(x, "T-statistic is ", statistic, "P value is " , pvalue)
#see code challenge 2 where you append to a dictionary  
ttest(dem, rep)

print(df.columns)
#function to look at all of the bills
#df.columns 

#def basic_statistics(bill):
  #ttest_ind(rep["water_project_cost_sharing"], dem["water_project_cost_sharing"], nan_policy="omit")
  

TypeError: ignored